The Trajectory Rotation Average $ (\mathrm{\overline{TRA}}_{t_0}^{t_N}) $ is computed from the velocity along the trajectories launched from initial conditions distributed over a regular meshgrid of size (Ny, Nx). The time-interval $ [t_0, t_N] $ is discretized with a resolution of N.

\begin{equation}
\mathrm{\overline{TRA}}_{t_0}^{t_N} = \dfrac{1}{t_N-t_0} \sum_{i = 0}^{N-1} \cos^{-1}\dfrac{\langle \mathbf{\dot{x}}(t_i), \mathbf{\dot{x}}(t_{i+1}) \rangle+\mathrm{v}_0^2}{\sqrt{|\mathbf{\dot{x}}(t_i)|^2+\mathrm{v}_0^2}\sqrt{|\mathbf{\dot{x}}(t_{i+1})|^2+\mathrm{v}_0^2}}
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| lenT | float | $ t_N-t_0 $ |
| v0 | float | $ \mathrm{v}_0 $|
| velocity_grid | array (Ny, Nx, Nz, 2, N) | velocity along trajectories launched from <br /> meshgrid of size (Ny, Nx, Nz) over the <br /> discretized time-interval with resolution N|
| TRA | array (Ny, Nx, Nz) | $ \mathrm{\overline{TRA}}_{t_0}^{t_N} $|

In [1]:
import numpy as np

# import Rectangular bivariate spline from scipy
from scipy.interpolate import RegularGridInterpolator as RGI

def _orient_vectorfield(X, Y, Z, x, vector_field):
            
        # Check for orientational discontinuity by introducing appropriate scaling
        idx_x = np.searchsorted(X[0, :, 0], x[0])
        idx_y = np.searchsorted(Y[:, 0, 0], x[1])
        idx_z = np.searchsorted(Z[0, 0, :], x[2])
        
        if idx_x == 0:
            idx_x += 1
        if idx_y == 0:
            idx_y += 1
        if idx_z == 0:
            idx_z += 1
        
        
        if 0 < idx_x < X.shape[1] and 0 < idx_y < Y.shape[0] and 0 < idx_z < Z.shape[2]:
    
            X_reduced = X[idx_y-1:idx_y+1, idx_x-1:idx_x+1, idx_z-1:idx_z+1]
            Y_reduced = Y[idx_y-1:idx_y+1, idx_x-1:idx_x+1, idx_z-1:idx_z+1]
            Z_reduced = Z[idx_y-1:idx_y+1, idx_x-1:idx_x+1, idx_z-1:idx_z+1]
        
            vx_grid = np.array([[[vector_field[idx_y-1,idx_x-1, idx_z-1, 0], vector_field[idx_y, idx_x-1, idx_z-1, 0]],
                      [vector_field[idx_y-1,idx_x, idx_z-1, 0], vector_field[idx_y, idx_x, idx_z-1, 0]]],
                      [[vector_field[idx_y-1,idx_x-1, idx_z, 0], vector_field[idx_y, idx_x-1, idx_z, 0]],
                      [vector_field[idx_y-1,idx_x, idx_z, 0], vector_field[idx_y, idx_x, idx_z, 0]]]])
            
            vy_grid = np.array([[[vector_field[idx_y-1,idx_x-1, idx_z-1, 1], vector_field[idx_y, idx_x-1, idx_z-1, 1]],
                      [vector_field[idx_y-1,idx_x, idx_z-1, 1], vector_field[idx_y, idx_x, idx_z-1, 1]]],
                      [[vector_field[idx_y-1,idx_x-1, idx_z, 1], vector_field[idx_y, idx_x-1, idx_z, 1]],
                      [vector_field[idx_y-1,idx_x, idx_z, 1], vector_field[idx_y, idx_x, idx_z, 1]]]])
            
            vz_grid = np.array([[[vector_field[idx_y-1,idx_x-1, idx_z-1, 2], vector_field[idx_y, idx_x-1, idx_z-1, 2]],
                      [vector_field[idx_y-1,idx_x, idx_z-1, 2], vector_field[idx_y, idx_x, idx_z-1, 2]]],
                      [[vector_field[idx_y-1,idx_x-1, idx_z, 2], vector_field[idx_y, idx_x-1, idx_z, 2]],
                      [vector_field[idx_y-1,idx_x, idx_z, 2], vector_field[idx_y, idx_x, idx_z, 2]]]])

            for i in range(2):
                for j in range(2):
                    for k in range(2):
                        if vx_grid[0, 0, 0]*vx_grid[i, j, k]+vy_grid[0, 0, 0]*vy_grid[i, j, k]+vz_grid[0, 0, 0]*vz_grid[i, j, k] < 0:
                            vx_grid[i, j, k] = -vx_grid[i, j, k]
                            vy_grid[i, j, k] = -vy_grid[i, j, k]
                            vz_grid[i, j, k] = -vz_grid[i, j, k]
    
            vx_Interp = RGI((Y_reduced[:, 0, 0], X_reduced[0, :, 0], Z_reduced[0, 0, :]), vx_grid)
            vy_Interp = RGI((Y_reduced[:, 0, 0], X_reduced[0, :, 0], Z_reduced[0, 0, :]), vy_grid)
            vz_Interp = RGI((Y_reduced[:, 0, 0], X_reduced[0, :, 0], Z_reduced[0, 0, :]), vz_grid)
        
            return vx_Interp(np.array([x[1], x[0], x[2]]))[0], vy_Interp(np.array([x[1], x[0], x[2]]))[0], vz_Interp(np.array([x[1], x[0], x[2]]))[0]
        
        else:
            
            return None, None, None